Chapter 8 Learning at Scale - Section 8.8 - Graph Coarsening

-------------------------------------------------------------------

This script demonstrates the process of graph coarsening using the Graclus
method with PyTorch Geometric. It starts with loading the KarateClub dataset,
a well-known social network dataset representing the relationships between
34 members of a karate club at a US university in the 1970s.

The goal is to perform graph coarsening to reduce the graph's complexity while
maintaining its essential structural and feature properties. The Graclus
method is used for clustering, followed by max-pooling to create a coarsened
graph. Node labels are also updated to reflect the most common label within
each cluster.

Our process below:
1. The KarateClub dataset is loaded.
2. Graclus clustering is performed on the graph.
3. New labels are generated for the coarsened graph.
4. An original training mask is assumed; if it doesn't exist, all nodes are considered for training.
5. A new training mask for the coarsened graph is created based on whether the clusters contain any training nodes from the original graph.
6. The new training mask is added to the coarsened data object.
7. The coarsened data object, with the new training mask, is printed for verification.

# Part I. Install Packages, Load Data and Import Packages

In [ ]:
# Find the CUDA version PyTorch was installed with
!python -c "import torch; print(torch.version.cuda)"

In [ ]:
# PyTorch version
!python -c "import torch; print(torch.__version__)"

In [ ]:
# Use the above information to fill in the http address below
# %%capture
!pip install ogb pyg-lib torch-scatter torch-cluster torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-geometric

In [ ]:
import torch
from torch_geometric.nn import graclus, max_pool
from torch_geometric.utils import to_undirected
from torch_geometric.datasets import KarateClub
from scipy import stats


##Coarsen the karate club dataset

In [ ]:

# Load dataset
dataset = KarateClub()
data = dataset[0]

# Print the original labels
print("Old Labels:")
print(data.y)

# Ensure edge_index is undirected for Graclus clustering
data.edge_index = to_undirected(data.edge_index)

"""
Perform clustering using the Graclus method. The Graclus algorithm is a
fast graph clustering method that maximizes the modularity over possible
clusterings. This section calculates clusters and prints them.
"""
cluster = graclus(data.edge_index, num_nodes=data.num_nodes)
cluster_n = cluster.to('cpu').numpy()
y_n = data.y.to('cpu').numpy()

"""
Generate new labels for the coarsened graph. For each cluster generated
by Graclus, find the most common label among the nodes in the cluster
and assign it as the new label for the coarsened node corresponding to
that cluster.
"""
new_labels = []
for i in range(cluster_n.max() + 1):
    labels_in_cluster = y_n[cluster_n == i]
    if labels_in_cluster.size > 0:
        most_common_label = stats.mode(labels_in_cluster).mode.item()
        new_labels.append(most_common_label)

# Convert new_labels to a tensor
new_labels = torch.tensor(new_labels, dtype=torch.long)

# Print the new labels
print("\nNew Labels:")
print(new_labels)

# Print the old data object
print("\nOld Data Object:")
print(data)

"""
Perform max pooling to obtain the coarsened graph, using the clusters
obtained from the Graclus algorithm. Max pooling helps to reduce the
graph size while retaining the essential structural and feature
characteristics.
"""
data_coarse = max_pool(cluster, data)
data_coarse.y = new_labels

# Assuming an original training mask exists; if not, consider all nodes for training
original_train_mask = data.train_mask if hasattr(data, 'train_mask') else torch.ones(data.num_nodes, dtype=bool)

# Print the original training mask
print("\nOriginal Training Mask:")
print(original_train_mask)

# Determine whether clusters contain any training nodes from the original graph
contains_training_node = torch.zeros(cluster_n.max() + 1, dtype=bool)
for i in range(cluster_n.max() + 1):
    nodes_in_cluster = cluster_n == i
    if original_train_mask[nodes_in_cluster].any().item():
        contains_training_node[i] = True

# Create a new training mask for the coarsened graph
new_train_mask = contains_training_node[new_labels.long()]

# Print the new training mask
print("\nNew Training Mask:")
print(new_train_mask)

# Add the new training mask to the coarsened data object
data_coarse.train_mask = new_train_mask

# Print the new coarsened data object with the training mask
print("\nNew Data Object with Training Mask:")
print(data_coarse)


##Coarsen the Amazon products dataset

In [ ]:
# importing obg datatset
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T

In [ ]:
import os.path as osp
root = osp.join(osp.dirname(osp.realpath('./')), 'data', 'products')


In [ ]:
dataset_dense = PygNodePropPredDataset( name='ogbn-products')
dataset_dense.num_classes

In [ ]:
%%time
# Load dataset
dataset = dataset_dense
data = dataset[0]

# Print the original labels
print("Old Labels:")
print(data.y)

# Ensure edge_index is undirected for Graclus clustering
data.edge_index = to_undirected(data.edge_index)

"""
Perform clustering using the Graclus method. The Graclus algorithm is a
fast graph clustering method that maximizes the modularity over possible
clusterings. This section calculates clusters and prints them.
"""
cluster = graclus(data.edge_index, num_nodes=data.num_nodes)
cluster_n = cluster.to('cpu').numpy()
y_n = data.y.to('cpu').numpy()

"""
Generate new labels for the coarsened graph. For each cluster generated
by Graclus, find the most common label among the nodes in the cluster
and assign it as the new label for the coarsened node corresponding to
that cluster.
"""
new_labels = []
for i in range(cluster_n.max() + 1):
    labels_in_cluster = y_n[cluster_n == i]
    if labels_in_cluster.size > 0:
        most_common_label = stats.mode(labels_in_cluster).mode.item()
        new_labels.append(most_common_label)

# Convert new_labels to a tensor
new_labels = torch.tensor(new_labels, dtype=torch.long)

# Print the new labels
print("\nNew Labels:")
print(new_labels)

# Print the old data object
print("\nOld Data Object:")
print(data)

"""
Perform max pooling to obtain the coarsened graph, using the clusters
obtained from the Graclus algorithm. Max pooling helps to reduce the
graph size while retaining the essential structural and feature
characteristics.
"""
data_coarse = max_pool(cluster, data)
data_coarse.y = new_labels

# Assuming an original training mask exists; if not, consider all nodes for training
original_train_mask = data.train_mask if hasattr(data, 'train_mask') else torch.ones(data.num_nodes, dtype=bool)

# Print the original training mask
print("\nOriginal Training Mask:")
print(original_train_mask)

# Determine whether clusters contain any training nodes from the original graph
contains_training_node = torch.zeros(cluster_n.max() + 1, dtype=bool)
for i in range(cluster_n.max() + 1):
    nodes_in_cluster = cluster_n == i
    if original_train_mask[nodes_in_cluster].any().item():
        contains_training_node[i] = True

# Create a new training mask for the coarsened graph
new_train_mask = contains_training_node[new_labels.long()]

# Print the new training mask
print("\nNew Training Mask:")
print(new_train_mask)

# Add the new training mask to the coarsened data object
data_coarse.train_mask = new_train_mask

# Print the new coarsened data object with the training mask
print("\nNew Data Object with Training Mask:")
print(data_coarse)
